In [1]:
import torch.nn as nn
import torch
class AutoEncoderConcat(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)
        self.relu1 = nn.ReLU()
        self.encoder2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.relu2 = nn.ReLU()
        self.encoder3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.relu3 = nn.ReLU()
        self.encoder4 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.relu4 = nn.ReLU()
        self.encoder5 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1)
        self.relu5 = nn.ReLU()
        self.drop1 = nn.Dropout(0.7)
        self.drop2 = nn.Dropout(0.7)
        self.drop3 = nn.Dropout(0.7)
        self.drop4 = nn.Dropout(0.7)

        self.decoder1 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2, padding=0)
        self.derelu1 = nn.ReLU()
        self.decoder2 = nn.ConvTranspose2d(512, 128, kernel_size=2, stride=2, padding=0)
        self.derelu2 = nn.ReLU()
        self.decoder3 = nn.ConvTranspose2d(256, 64, kernel_size=2, stride=2, padding=0)
        self.derelu3 = nn.ReLU()
        self.decoder4 = nn.ConvTranspose2d(128, 32, kernel_size=2, stride=2, padding=0)
        self.derelu4 = nn.ReLU()
        self.decoder5 = nn.ConvTranspose2d(64, 3, kernel_size=2, stride=2, padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x1 = self.relu1(self.encoder1(x))
        # print(x1.shape)
        x2 = self.relu2(self.encoder2(x1))  
        # print(x2.shape)
        x3 = self.relu3(self.encoder3(x2))  
        # print(x3.shape)
        x4 = self.relu4(self.encoder4(x3))  
        # print(x4.shape)
        x5 = self.relu5(self.encoder5(x4))  
        # print(x5.shape)

        d1 = self.derelu1(self.decoder1(x5))
        # print(d1.shape)
        d2 = self.derelu2(self.decoder2(torch.concatenate([d1,self.drop1(x4)], dim=1)))
        # print(d2.shape)
        d3 = self.derelu3(self.decoder3(torch.concatenate([d2,self.drop2(x3)], dim=1)))
        # print(d3.shape)
        d4 = self.derelu4(self.decoder4(torch.concatenate([d3,self.drop3(x2)], dim=1)))
        # print(d4.shape)
        decoded = self.sigmoid(self.decoder5(torch.concatenate([d4 ,self.drop4(x1)], dim=1)))
        # print(decoded.shape)
        x = decoded 

        return x
    


In [2]:
model1 = AutoEncoderConcat() 
model1.load_state_dict(torch.load('AutoencoderConcat_drop_256.pth', map_location=torch.device('cpu')))
model1.to('cpu') 
# model = model1.to(torch.bfloat16)

/tmp/ipykernel_10439/2783876366.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1.load_state_dict(torch.load('AutoencoderConcat_drop_256.pth', map_location=torch.de

AutoEncoderConcat(
  (encoder1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu1): ReLU()
  (encoder2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu2): ReLU()
  (encoder3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu3): ReLU()
  (encoder4): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu4): ReLU()
  (encoder5): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu5): ReLU()
  (drop1): Dropout(p=0.7, inplace=False)
  (drop2): Dropout(p=0.7, inplace=False)
  (drop3): Dropout(p=0.7, inplace=False)
  (drop4): Dropout(p=0.7, inplace=False)
  (decoder1): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2))
  (derelu1): ReLU()
  (decoder2): ConvTranspose2d(512, 128, kernel_size=(2, 2), stride=(2, 2))
  (derelu2): ReLU()
  (decoder3): ConvTranspose2d(256, 64, kernel_size=(2, 2), stride=(2, 2))
  (derelu3): ReLU()
  (decoder4): ConvTranspose2d(128, 32, 

In [3]:
import argparse
import sys
import os
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image
from PIL import Image
from models import Generator, Discriminator, FeatureExtractor

device = 'cpu'  
IMAGE_SIZE = 32
BATCH_SIZE = 1

if torch.cuda.is_available():
    print("WARNING: You have a CUDA device, but this script is set to run on the CPU.")

transform = transforms.Compose([transforms.ToTensor()])

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

scale = transforms.Compose([transforms.ToPILImage(),
                            transforms.Resize(IMAGE_SIZE),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                 std=[0.229, 0.224, 0.225])
                            ])

unnormalize = transforms.Normalize(mean=[-2.118, -2.036, -1.804], std=[4.367, 4.464, 4.444])

generator = Generator(16, 4)
# cange the dir 
generator.load_state_dict(torch.load('checkpoints/generator_pretrain.pth', map_location=device))  


generator.to(device)

low_res = torch.FloatTensor(BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE).to(device)
print('Test started...')
mean_generator_content_loss = 0.0
mean_generator_adversarial_loss = 0.0
mean_generator_total_loss = 0.0
mean_discriminator_loss = 0.0


generator.eval()



Test started...


/tmp/ipykernel_10439/866384718.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load('checkpoints/generator_pretrain.pth', map_location=d

Generator(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (residual_block1): residualBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (residual_block2): residualBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (residual_block3): residualBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, 

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from sklearn.metrics import f1_score, accuracy_score, recall_score
import os
from torch.utils.data import random_split

class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 32 * 32, 512) 
        self.fc2 = nn.Linear(512, 2) 
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.0)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 32 * 32)  
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [6]:
import os
from numpy.fft import fft2, fftshift
import cv2
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
import shutil

input_folder = 'perturbed_images_32'  
output_folder_fake = 'fake_images'  

os.makedirs(output_folder_fake, exist_ok=True)

transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.ToTensor(),        
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform2 = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  
])

# Load model
model = CNNClassifier()
model.load_state_dict(torch.load('classifier_all_imag_epoch_12.pth', map_location='cpu'))  # Load weights from the last epoch
model.eval()  

device = torch.device("cpu")  
model.to(device)


image_count = 0
real_count = 0
fake_count = 0

for root, dirs, files in os.walk(input_folder):
    for image_file in files:
        if image_file.endswith(('png', 'jpg', 'jpeg')):
            image_path = os.path.join(root, image_file)

            image = Image.open(image_path).convert('RGB')
            low_res = scale(np.array(image)).reshape(-1, 3, 32, 32)

            high_res_fake = generator(Variable(low_res))
            high_res_fake = generator(Variable(high_res_fake))
            high_res = []
            for j in range(len(high_res_fake)):
                high_res.append(transforms.Resize(256)(unnormalize(high_res_fake[j])))
            high_res_fake = torch.stack(high_res)

            reconstructed = model1(high_res_fake) 
            noise = high_res_fake - reconstructed

            gray = cv2.cvtColor(noise[0].cpu().detach().permute(1, 2, 0).numpy(), cv2.COLOR_RGB2GRAY)

            fourier_spectrum = fftshift(fft2(gray))
            magnitude_spectrum = torch.abs(torch.tensor(fourier_spectrum))

            log_magnitude_spectrum = np.log1p(magnitude_spectrum.numpy())

            output_image_path = 'log_magnitude_spectrum.png'
            plt.imsave(output_image_path, log_magnitude_spectrum, cmap='gray')

            img_path = output_image_path
            img = Image.open(img_path)

            img_tensor = transform2(img).unsqueeze(0).to(device)  

            with torch.no_grad():
                output = model(img_tensor)
                _, predicted_class = torch.max(output, 1)

            class_mapping = {0: "real", 1: "fake"}
            predicted_label = class_mapping[predicted_class.item()]

            if predicted_label == "fake":
                fake_count += 1
                print(f"Fake image detected: {image_file}")

                shutil.copy(img_path, os.path.join(output_folder_fake, image_file))  

            image_count += 1

            

print("Processing completed for all images.")
print(f"Real images processed: {image_count-real_count}")
print(f"Fake images processed: {fake_count}")


/tmp/ipykernel_10439/1368018383.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('classifier_all_imag_epoch_12.pth', map_location='cpu')

Fake image detected: 141.png
Fake image detected: 109.png
Fake image detected: 241.png
Fake image detected: 8.png
Fake image detected: 71.png
Fake image detected: 123.png
Fake image detected: 273.png
Fake image detected: 2.png
Fake image detected: 93.png
Fake image detected: 107.png
Fake image detected: 3.png
Fake image detected: 292.png
Fake image detected: 206.png
Fake image detected: 125.png
Fake image detected: 143.png
Fake image detected: 169.png
Fake image detected: 12.png
Fake image detected: 180.png
Fake image detected: 147.png
Fake image detected: 189.png
Fake image detected: 144.png
Fake image detected: 96.png
Fake image detected: 25.png
Fake image detected: 37.png
Fake image detected: 289.png
Fake image detected: 111.png
Fake image detected: 39.png
Fake image detected: 58.png
Fake image detected: 188.png
Fake image detected: 134.png
Fake image detected: 21.png
Fake image detected: 218.png
Fake image detected: 66.png
Fake image detected: 249.png
Fake image detected: 284.png
F